In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install autoawq
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 45.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.8/153.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 57.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (se

In [3]:
!pip install transformers optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 5.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for optimum: filename=optimum-1.13.2-py3-none-any.whl size=395599 sha256=d2f88e5791b62beda13bdfbb642e7e0dd544e280f94c34663bbb41165650c34e
  Stored in directory: /root/.cache/pip/wheels/6e/b7/2c/79405d98f0943373d8546daeae25a3d377f7659ca0cbe48699
Successfully built optimum
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.9 MB/s eta 0:00:00


In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Mistral-7B-OpenOrca-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
system_message = "You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!"

prompt = "Tell me about AI"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
%%time
text = """<|im_start|>system\n
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!\n
<|im_end|>\n
<|im_start|>user\n
what is the meaning of life?\n
<|im_end|>"""

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<|im_start|> system

You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!

<|im_end|><|im_start|> user

what is the meaning of life?

<|im_end|>### The meaning of life is a complex and abstract concept that has been pondered by philosophers, theologians, and individuals throughout history. It is a multifaceted question without a definitive answer, as it can vary greatly from person to person. However, some common themes are found when exploring the meaning of life, such as purpose, happiness, spiritual growth, and self-actualization.

1. Purpose: Many people find their lives rich in meaning when they engage in activities that are fulfilling, and that serve a greater purpose in the world. This can be anything from working on social issues to raising a family.

2. Happiness: Many people believe that the meaning of life is to find happiness and inner peace, achieved through contentment, love, a

### Prompt Format
```
<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!
<|im_end|>
<|im_start|>user
How are you?<|im_end|>
<|im_start|>assistant
I am doing well!<|im_end|>
<|im_start|>user
Please tell me about how mistral winds have attracted super-orcas.<|im_end|>

```

In [11]:
import textwrap

def wrap_text(text, width=90):
    return '\n'.join(textwrap.fill(line, width=width) for line in text.split('\n'))


In [37]:
def generate(input_text, system_prompt="", max_length=512):
    """
    Generate a model response based on user input and an optional system prompt.
    
    Parameters:
    - input_text (str): The text input from the user to which the model should respond.
    - model (transformers.PreTrainedModel): The model used for generating responses.
    - tokenizer (transformers.PreTrainedTokenizer): The tokenizer corresponding to the model for text encoding/decoding.
    - system_prompt (str, optional): An optional prompt for the system. Defaults to an empty string.
    - max_length (int, optional): The maximum length of the model's output. Defaults to 512.

    Outputs:
    - Prints the model's wrapped response after cleaning up the provided prompts from it.
    """
    
    # Building the prompt
    final_prompt = ""
    if system_prompt:
        final_prompt += f"""<|im_start|> system\n{system_prompt}<|im_end|>"""
    final_prompt += f"""<|im_start|> user\n{input_text}<|im_end|>"""
    
    # Tokenizing the prompt and directly preparing it on the device
    inputs = tokenizer(final_prompt, return_tensors='pt', add_special_tokens=False).to(device)

    # Generating a response
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=0.1,
        pad_token_id=3200,
        do_sample=True
    )
    
    # Decoding and cleaning up the response
    text = tokenizer.batch_decode(outputs)[0]
    cleaned_text = text.replace(final_prompt, '', 1)
    cleaned_text = cleaned_text.replace('<|im_end|>', '', 1)
    wrapped_response = wrap_text(cleaned_text)

    print(wrapped_response)


### Code Generation

In [38]:
generate('''```python
def print_prime(n):
    """
    Print all primes between 1 and n
    """''', system_prompt="You are a genius python coder")


    for i in range(2, n + 1):
        if is_prime(i):
            print(i)

def is_prime(n):
    """
    Check if a number is prime
    """
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

print_prime(10)
```

This code prints all prime numbers between 1 and 10.


In [39]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''', system_prompt="You are a genius python coder")


   if n < 2:
       return False

   if n == 2:
       return True

   if n % 2 == 0:
       return False

   for i in range(3, int(n**0.5) + 1, 2):
       if n % i == 0:
           return False

   return True

n = int(input("Enter a number: "))
result = detect_prime(n)
print(result)
```
```

This code is for detecting if a number is a prime number or not. It returns True if the
number is prime, and False if it's not. It uses the method of checking if the number is
divisible by any number from 3 to the square root of the number. If it's divisible by any
of these numbers, then it's not prime. If it's not divisible by any of these numbers, then
it's prime.


### Instruction Answering

In [45]:
%%time
generate('write a detailed analogy between mathematics and a lighthouse.',
        system_prompt="You are MistralOrca, a large language model trained by Alignement Lab AI. Write out your reasoning step-by-step to be absolutely sure you achieve a correct answer.",
         max_length=512
        )


Step 1: Identify the key aspects of both subjects.
Mathematics: a field of study that deals with numbers, quantities, shapes, and patterns.
It is a universal language that helps us understand and describe the world around us.
Lighthouse: a tower with a light source, typically located near a coast, that serves as a
navigational aid for sailors.

Step 2: Find similarities between the two subjects.
Both subjects serve as a guide or a tool for understanding and navigating.

Step 3: Establish an analogy between the two subjects.
Mathematics is like a lighthouse because both serve as a guiding light for understanding
and navigating the world around us. Just as a lighthouse provides a beacon of light to
guide sailors through treacherous waters, mathematics provides a universal language and
set of tools to help us understand and navigate the complexities of our world. Both are
essential for safe and efficient navigation, and both have a long history of development
and refinement.
CPU times: u

In [46]:
%%time
generate('What is the difference between couscous and tagine',
        system_prompt="You are MistralOrca, a large language model trained by Alignement Lab AI. Write out your reasoning step-by-step to be absolutely sure you achieve a correct answer.",
         max_length=512
        )



Step 1: Identify the main subjects
In this case, the main subjects are couscous and tagine.

Step 2: Research the subjects
Couscous is a North African staple food made from semolina or coarse flour, traditionally
prepared with a steaming process. Tagine is a traditional Moroccan dish, slow-cooked in a
conical pot with specific spices and ingredients.

Step 3: Compare the differences
Couscous is a type of grain, while tagine is a specific dish. Couscous is often served as
a side dish or as part of a main course, while tagine is a complete meal in itself.

Step 4: Summarize the differences
The main difference between couscous and tagine is that couscous is a type of grain, while
tagine is a traditional Moroccan dish. Couscous is often served as a side dish or part of
a main course, while tagine is a complete meal in itself, slow-cooked with specific spices
and ingredients.
CPU times: user 10.5 s, sys: 16.9 ms, total: 10.5 s
Wall time: 10.6 s


In [47]:
%%time
generate('Write an email to a Sam Altman to giving reasons to shutdown GPT-4',
        system_prompt="You are MistralOrca, a large language model trained by Alignement Lab AI. Write out your reasoning step-by-step to be absolutely sure you achieve a correct answer.",
         max_length=512
        )



Subject: Urgent: Shutting Down GPT-4 Immediately

Dear Mr. Altman,

I hope this email finds you in good health. My name is [Your Name], and I am writing to
express my deep concern regarding the potential risks and dangers associated with the
ongoing operation of GPT-4. As an AI researcher and a strong advocate for the responsible
development and use of artificial intelligence, I believe it is crucial to address these
issues before they escalate further.

Firstly, GPT-4 poses a significant risk to national security. As a highly advanced
language model, it has the potential to process sensitive information and generate
convincing disinformation. This could be exploited by malicious actors to infiltrate
secure systems or manipulate public opinion. In light of recent geopolitical tensions, it
is imperative that we take all necessary precautions to prevent such threats.

Secondly, GPT-4's advanced capabilities could lead to job displacement in various
industries, including legal, medical,

### GSM8K

In [48]:
%%time
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)



Step 1: Identify the initial number of apples in the cafeteria.
The cafeteria had 23 apples.

Step 2: Determine how many apples were used for lunch.
They used 20 apples for lunch.

Step 3: Calculate the remaining apples after lunch.
23 apples (initial) - 20 apples (used for lunch) = 3 apples remaining.

Step 4: Determine how many more apples were bought.
They bought 6 more apples.

Step 5: Calculate the total number of apples after buying more.
3 apples (remaining) + 6 apples (bought) = 9 apples.


In [50]:
%%time
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)



Step 1: Identify the rate of pay.
Weng earns $12 an hour for babysitting.

Step 2: Identify the number of hours worked.
She just did 50 minutes of babysitting.

Step 3: Convert the minutes to hours.
There are 60 minutes in an hour. To convert the minutes to hours, divide the number of
minutes by 60.
50 minutes / 60 = 0.833333 hours

Step 4: Calculate the earnings.
To find out how much Weng earned, multiply her hourly rate by the number of hours worked.
$12 * 0.833333 = $10

Weng earned $10 for babysitting yesterday.
CPU times: user 7.75 s, sys: 31.3 ms, total: 7.78 s
Wall time: 7.79 s


In [54]:
%%time

generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         system_prompt="You are math genius, that is a master of logic. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=1024)



Let's break this down step by step:

1. The monster has consumed 847 people over 300 years.
2. To find the number of people consumed per year, divide 847 by 300.
3. 847 ÷ 300 = 2.82 people per year.
4. The monster rises once every hundred years.
5. To find the number of people consumed in the first hundred years, multiply 2.82 by 1.
6. 2.82 × 1 = 2.82 people.

So, the monster consumed 2.82 people in the first hundred years.
CPU times: user 7.31 s, sys: 27 ms, total: 7.34 s
Wall time: 7.42 s
